把概念都记一下吧，用于以后复习。
- FP，依次计算并存储NN的中间变量。
- BP，依次计算并存储NN的中间变量和参数的**梯度**。
- DNN中FP，BP相互依赖，FP和BP交替，FP结束结果一般不释放，用作下次BP

# 正向传播（FP）
输入特征$\boldsymbol{x} \in \mathbb{R}^{d}$，

则**中间变量**：$\boldsymbol{z}=\boldsymbol{W}^{(1)} \boldsymbol{x}$，

其中，$\boldsymbol{W}^{(1)} \in \mathbb{R}^{h \times d}$，将中间变量$\boldsymbol{z} \in \mathbb{R}^{h}$经过激活函数后，得到

**隐藏层变量**：$\boldsymbol{h}=\phi(\boldsymbol{z})$

同理，$\boldsymbol{W}^{(2)} \in \mathbb{R}^{q \times h}$，则$\boldsymbol{o} \in \mathbb{R}^{q}$

**输出层变量**：$\boldsymbol{o}=\boldsymbol{W}^{(2)} \boldsymbol{h}$

假设损失函数为$\ell$,样本标签为$y$则

损失函数结果（**损失项**）为：$L=\ell(\boldsymbol{o}, y)$

正则化项（**惩罚项**）：$s=\frac{\lambda}{2}\left(\left\|\boldsymbol{W}^{(1)}\right\|_{F}^{2}+\left\|\boldsymbol{W}^{(2)}\right\|_{F}^{2}\right)$

则带上惩罚项的损失项为：$J=L+s$

$J$就是给定数据样本的**目标函数**

# 反向传播（BP）
对于任意形状张量（Tensor）的函数$Y=f(X)$和$Z=g(Y)$，通过链式法则，有：
$$
\frac{\partial \mathrm{Z}}{\partial \mathrm{X}}=\operatorname{prod}\left(\frac{\partial \mathrm{Z}}{\partial \mathrm{Y}}, \frac{\partial \mathrm{Y}}{\partial \mathrm{X}}\right)
$$
prob运算，根据两个输入，在有必要的操作（转置、换位）后对二者做乘法。

拿前面FP举例，目标函数$J$对输出层变量$o$的梯度：$\delta J / \partial \boldsymbol{o} \in \mathbb{R}^{q}$：

$$
\frac{\partial J}{\partial \boldsymbol{o}}=\operatorname{prod}\left(\frac{\partial J}{\partial L}, \frac{\partial L}{\partial \boldsymbol{o}}\right)=\frac{\partial L}{\partial \boldsymbol{o}}
$$

其中，$J=L+s$，$\frac{\partial J}{\partial L}=1$故有上式

目标函数$J$对输出层变量$W^(2)$的梯度：$\partial J / \partial \boldsymbol{W}^{(2)} \in \mathbb{R}^{q \times h}$：

$$
\frac{\partial J}{\partial \boldsymbol{W}^{(2)}}=\operatorname{prod}\left(\frac{\partial J}{\partial \boldsymbol{o}}, \frac{\partial \boldsymbol{o}}{\partial \boldsymbol{W}^{(2)}}\right)+\operatorname{prod}\left(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial \boldsymbol{W}^{(2)}}\right)=\frac{\partial J}{\partial \boldsymbol{o}} \boldsymbol{h}^{\top}+\lambda \boldsymbol{W}^{(2)}
$$

沿着输出层向隐藏层继续反向传播，$\partial J / \partial \boldsymbol{h} \in \mathbb{R}^{h}$，则：

$$
\frac{\partial J}{\partial \boldsymbol{h}}=\operatorname{prod}\left(\frac{\partial J}{\partial \boldsymbol{o}}, \frac{\partial \boldsymbol{o}}{\partial \boldsymbol{h}}\right)=\boldsymbol{W}^{(2) \top} \frac{\partial J}{\partial \boldsymbol{o}}
$$

激活函数是按元素运算符的，中间变量$z$的梯度$\partial J / \partial z \in \mathbb{R}^{h}$的计算需要使用按元素乘法符$\odot$：

$$
\frac{\partial J}{\partial z}=\operatorname{prod}\left(\frac{\partial J}{\partial \boldsymbol{h}}, \frac{\partial \boldsymbol{h}}{\partial \boldsymbol{z}}\right)=\frac{\partial J}{\partial \boldsymbol{h}} \odot \phi^{\prime}(\boldsymbol{z})
$$

最终得到靠近输入层权重的梯度$\partial J / \partial \boldsymbol{W}^{(1)} \in \mathbb{R}^{h \times d}$，则：

$$
\frac{\partial J}{\partial \boldsymbol{W}^{(1)}}=\operatorname{prod}\left(\frac{\partial J}{\partial \boldsymbol{z}}, \frac{\partial \boldsymbol{z}}{\partial \boldsymbol{W}^{(1)}}\right)+\operatorname{prod}\left(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial \boldsymbol{W}^{(1)}}\right)=\frac{\partial J}{\partial \boldsymbol{z}} \boldsymbol{x}^{\top}+\lambda \boldsymbol{W}^{(1)}
$$
其中，$J=L+s$，$\frac{\partial J}{\partial s}=1$故有上式

# 梯度爆炸、梯度消失

当NN的层数较多时，模型的稳定性容易会变差。
$$
\boldsymbol{H}^{(l)}=\boldsymbol{X} \boldsymbol{W}^{(1)} \boldsymbol{W}^{(2)} \ldots \boldsymbol{W}^{(l)}
$$
如果层数为30层，考虑两种情况，权重参数为0.2与5，则X分别与$0.2^{30} \approx 1 \times 10^{-21}$和$5^{30} \approx 9 \times 10^{20}$，前者为衰减、后者为爆炸。